# Notebook for Splunk Machine Learning Toolkit Container for TensorFlow

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk Machine Learning Toolkit (MLTK) Container for TensorFlow. This script contains an example of how to run an entity extraction algorithm over text using the spacy library.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import spacy

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("spacy version: " + spacy.__version__)

In [ ]:
import sys
!{sys.executable} -m spacy download en_core_web_sm

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| makeresults
| eval text = "Boris Johnson has met Emmanuel Macron in Paris for Brexit talks, with the French president saying the UK's vote to quit the EU must be respected, but he added that the Ireland-Northern Ireland backstop plan was 'indispensable' to preserving political stability and the single market.;The backstop, opposed by Mr Johnson, aims to prevent a hard border on the island of Ireland after Brexit. Mr Johnson said that with 'energy and creativity we can find a way forward'.;On Wednesday German Chancellor Angela Merkel said the onus was on the UK to find a workable plan.;UK Prime Minister Mr Johnson insists the backstop must be ditched if a no-deal exit from the EU on 31 October is to be avoided.;He argues that it could leave the UK tied to the EU indefinitely, contrary to the result of the 2016 referendum, in which 52% of voters opted to leave.;But the EU has repeatedly said the withdrawal deal negotiated by former PM Theresa May, which includes the backstop, cannot be renegotiated.;However, it has previously said it would be willing to 'improve' the political declaration - the document that sets out the UK's future relationship with the EU.;Speaking after he greeted Mr Johnson at Paris's Elysee Palace, Mr Macron said he was 'very confident' that the UK and EU would be able to find a solution within 30 days - a timetable suggested by Mrs Merkel - 'if there is a good will on both sides'.;He said it would not be possible to find a new withdrawal agreement 'very different from the existing one' within that time, but added that an answer could be reached 'without reshuffling' the current deal.;Mr Macron also denied that he was the 'hard boy in the band', following suggestions that he would be tougher on the UK than his German counterpart.;Standing beside Mr Macron, Mr Johnson said he had been 'powerfully encouraged' by his conversations with Mrs Merkel in Berlin on Wednesday.;He emphasised his desire for a deal with the EU but added that it was 'vital for trust in politics' that the UK left the EU on 31 October.'He also said that 'under no circumstances' would the UK put checks or controls on the Ireland-UK border.;The two leaders ate lunch, drank coffee and walked through the Elysee gardens together during their talks, which lasted just under two hours. Mr Johnson then left to fly back to the UK."
| makemv text delim=";"
| mvexpand text
| fit MLTKContainer algo=spacy_ner epochs=100 text into app:spacy_entity_extraction_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("spacy_entity_extraction_model in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("spacy_entity_extraction_model")
print(df)
print(df.shape)
print(str(param))

## Stage 2 - create and initialize a model

In [ ]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    # Load English tokenizer, tagger, parser, NER and word vectors
    import en_core_web_sm
    model = en_core_web_sm.load()
    #model = spacy.load("en_core_web_sm")
    return model

In [ ]:
model = init(df,param)

## Stage 3 - fit the model

Note that for this algorithm the model is pre-trained (the en_core_web_sm library comes pre-packaged by spacy) and therefore this stage is a placeholder only

In [ ]:
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    
    return returns

## Stage 4 - apply the model

In [ ]:
def apply(model,df,param):
    X = df[param['feature_variables']].values.tolist()
    
    returns = list()
    
    for i in range(len(X)):
        doc = model(str(X[i]))
        
        
        entities = ''
    
        # Find named entities, phrases and concepts
        for entity in doc.ents:
            if entities == '':
                entities = entities + entity.text + ':' + entity.label_
            else:
                entities = entities + '|' + entity.text + ':' + entity.label_
        
        returns.append(entities)
    return returns

In [ ]:
returns = apply(model,df,param)
print(returns)

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return model summary
def summary(model=None):
    returns = {"version": {"spacy": spacy.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code